## Task-4 : Proxy Integration and Dynamic Handling 

The IMDB website was accessed and scraped, utilizing proxies due to its restrictive scraping policies. Two distinct pages were targeted for scraping, employing a combination of Selenium and BeautifulSoup. Subsequently, the scraped data was consolidated into a dataframe for convenient storage and analysis.

### NOTE
-> This assessment is mainly focused on Proxy integration. so, error handling and some potentional delays are not considered.

->The code runs only if user is using firebox as browser.Code need to be changed according to browser

In [1]:
#importing required libraries
import random
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#list of proxies from proxyscrape.com
proxies=['20.235.159.154:80',
'20.235.159.154:3128',
'103.159.90.14:83',
'114.143.0.177:80',
'117.250.3.58:8080',
'149.129.176.63:6666',
'210.212.39.138:8080']

In [3]:
#creating a proxy function for setting up a proxy webdriver
def proxy():
    return random.choice(proxies)

In [4]:
#setting up webdriver forfirefox
#updating path to geckodriver
options=Options()
options.add_argument(f'--proxy-server=http://{proxy}')
service=Service(r'C:\users\asuna\Downloads\geckodriver.exe')
driver= webdriver.Firefox(service=service,options=options)
driver.maximize_window()
url='https://www.imdb.com/'
driver.get(url)

In [5]:
#Automating path to Menu

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ipc-responsive-button')))
menu_bar = driver.find_elements(By.CLASS_NAME, 'ipc-responsive-button')
menu_bar[0].click()
print('Entered Menu')

Entered Menu


In [6]:
#Automating path to required table

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, 'Top 250 Movies')))
film = driver.find_elements(By.LINK_TEXT, 'Top 250 Movies')
film[0].click()
print('Top 250 movies list is visble')
time.sleep(10)

Top 250 movies list is visble


In [7]:
#alternative path for navigating towards top 250 shows and scraping top 250 movies
'''
#nativigating towards top 250 shows button before scraping movie titles
element = driver.find_element(By.LINK_TEXT,'Top 250 TV Shows')
driver.execute_script("arguments[0].scrollIntoView();", element)

#Scraping movie titles using selinium
film=driver.find_elements(By.CLASS_NAME,'ipc-title-link-wrapper')
movies=[movie.text.strip() for movie in film]
len(movies)
#NOTE: length need to be 258 if not wait for page to reload

element.click()
time.sleep(10)

'''

'\n#nativigating towards top 250 shows button before scraping movie titles\nelement = driver.find_element(By.LINK_TEXT,\'Top 250 TV Shows\')\ndriver.execute_script("arguments[0].scrollIntoView();", element)\n\n#Scraping movie titles using selinium\nfilm=driver.find_elements(By.CLASS_NAME,\'ipc-title-link-wrapper\')\nmovies=[movie.text.strip() for movie in film]\nlen(movies)\n#NOTE: length need to be 258 if not wait for page to reload\n\nelement.click()\ntime.sleep(10)\n\n'

In [8]:
#Scraping movie titles using selinium
film=driver.find_elements(By.CLASS_NAME,'ipc-title-link-wrapper')
movies=[movie.text.strip() for movie in film]
len(movies)
#NOTE: length need to be 258 if not wait for page to reload

258

In [9]:
#Navgating path back to Menu
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ipc-responsive-button')))
menu_bar = driver.find_elements(By.CLASS_NAME, 'ipc-responsive-button')
menu_bar[0].click() #clicking on menu button
print('Entered Menu')

Entered Menu


In [10]:
#Automating path to Top 250 shows list
time.sleep(3)
WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.LINK_TEXT, 'Top 250 TV Shows')))
TV = driver.find_elements(By.LINK_TEXT, 'Top 250 TV Shows')
TV[0].click()
print('Top 250 Shows list is visble')
time.sleep(10)

Top 250 Shows list is visble


In [11]:
page_source=driver.page_source
driver.quit()
soup=BeautifulSoup(page_source,'html.parser')
#Scraping TV Shows Titles using beautiful soup
shows=soup.find_all(class_='ipc-title-link-wrapper')
# Making a list of Shows
Shows=[show.text for show in shows]
len(Shows)
#NOTE: length need to be 258 if not wait for page to reload

258

handling the Data Frame

In [12]:
#Creating a dataframe

Top_250=pd.DataFrame({'Movies':movies,'Shows':Shows})

In [13]:
#There need to be only 250 rows so remaining row need to be removed
Top_250=Top_250[:250]

In [14]:
Top_250.tail()

,Movies,Shows
245,246. It Happened One Night,246. Your Lie in April
246,247. Drishyam,247. Community
247,248. Gangs of Wasseypur,248. Tear Along the Dotted Line
248,249. Dances with Wolves,249. Chef's Table
249,250. Aladdin,250. Foyle's War


In [15]:
#Creating a function to remove numbers before the titles
def col(x):
    c=x.split('. ')
    p=c[1]
    return p
Top_250['Movies']=Top_250['Movies'].apply(lambda x:col(x))
Top_250['Shows']=Top_250['Shows'].apply(lambda x:col(x))

In [16]:
#Final Table
Top_250.head()

,Movies,Shows
0,The Shawshank Redemption,Breaking Bad
1,The Godfather,Planet Earth II
2,The Dark Knight,Planet Earth
3,The Godfather Part II,Band of Brothers
4,12 Angry Men,Chernobyl
